[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LianjiaTech/BELLE/blob/main/notebook/BELLE_INFER_COLAB.ipynb) 

# ** BELLE模型在COLAB推理的示例** (目前由于机器配置，只有colab pro账户才能成功运行，正在优化中)
这里提供在colab环境运行BELLE模型的代码。默认加载的是8bit量化的BLOOM 7B模型，在模型加载到内存过程中，最高消费RAM大概需要20G，等模型load到GPU中以后，RAM只需要10G，GPU大概需要10G。 目前colab环境免费账户RAM 只有12G，所以暂时无法运行。需要将runtime -> change runtime type ->Runtime shape->High RAM.然后运行。


## 查看colab分配的显卡类型，一般免费账户上14G的T4显卡



---



In [ ]:
!nvidia-smi

Wed Apr 12 13:27:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##  将BELLE项目git clone到colab

In [ ]:
!git clone https://github.com/LianjiaTech/BELLE.git 


fatal: destination path 'BELLE' already exists and is not an empty directory.


### 14G显卡目前只支持量化版本，这里暂时只提供量化版本在colab推理

In [ ]:
%cd BELLE/gptq

/content/BELLE/gptq


### 安装gptq环境

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_uy73j4z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_uy73j4z
  Resolved https://github.com/huggingface/transformers to commit 370f0ca18c8e4577357df59936e790acdecef4ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
! python setup_cuda.py install && CUDA_VISIBLE_DEVICES=0 && python test_kernel.py


running install
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing quant_cuda.egg-info/PKG-INFO
writing dependency_links to quant_cuda.egg-info/dependency_links.txt
writing top-level names to quant_cuda.egg-info/top_level.txt
/usr/local/lib/python3.9/dist-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
reading manifest file 'quant_cuda.egg-info/SOURCES.txt'
writing manifest file 'quant_cuda.

### 将BELLE-7B-gptq版本下载到colab


In [ ]:

!git lfs install && git clone https://huggingface.co/BelleGroup/BELLE-7B-gptq

Updated git hooks.
Git LFS initialized.
fatal: destination path 'BELLE-7B-gptq' already exists and is not an empty directory.


In [ ]:
!ls BELLE-7B-gptq

bloom7b-0.2m-4bit-128g.pt  bloom7b-2m-8bit-128g.pt  special_tokens_map.json
bloom7b-0.2m-8bit-128g.pt  config.json		    tokenizer_config.json
bloom7b-2m-4bit-128g.pt    README.md		    tokenizer.json


In [ ]:
!pwd

/content/BELLE/gptq


##   运行下面的cell，然后在Human后面点击输入你和BELLE的对话内容。

In [ ]:
! python bloom_inference.py BelleGroup/BELLE-7B-1M  --wbits 8 --groupsize 128 --load  BELLE-7B-gptq/bloom7b-2m-8bit-128g.pt

Loading model ...
Done.
Human:
你是谁？
Assistant:

2023-04-12 13:34:31.490170: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Human: 你是谁？

Assistant: 我是一台计算机程序，名叫Belle，是法语单词“漂亮”翻译成英文的缩写。</s>

-------------------------------

写一首歌
Assistant:

Human: 写一首歌

Assistant:

Verse 1:
I was a hermit in the world
Lost in my own thoughts
But then you came along
And made my life complete

Chorus:
You make me feel alive
With your love I can飞
I never thought I'd find
What I was looking for

Verse 2:
I was a stranger in my own country
Wondering who I was
But then I met you
And my heart began to sing

Chorus:
You make me feel alive
With your love I can飞
I never thought I'd find
What I was looking for

Bridge:
I never thought I'd love again
But when I saw you, I knew I had to try
I never thought I'd find
A love that I could trust

Chorus:
You make me feel alive
With your love I can飞
I never thought I'd find
What I was looking for

Outro:
I never thought I'd 